# PageRank

In this notebook, we'll explore a basic application of PageRank using PySpark. Before diving into the implementation, let's review the theory behind PageRank.

Suppose we have a graph with nodes and edges, similar to the structure of the web. Our goal is to determine the importance of each node in the graph. In the context of the web, this translates to ranking web pages based on their relevance. However, we can't simply ask people to directly rank pages. Instead, we must utilize the information already available to us. Let's consider an example of a small web graph to illustrate this concept.

<img src="https://miro.medium.com/max/892/1*E1QqUL6eQpJsuxI9V5FE7g.png" alt="alt text" width="400"/>

Here we have 5 web pages, denoted as ${0, 1, 2, 3, 4}$. Each edge $i \rightarrow j$ signifies that web page $i$ refers to web page $j$. One initial approach to determine the relevance of a web page could be to consider the number of other web pages referring to it. For instance, if a page receives many referrals, it may be assumed to be more relevant. However, this approach fails to account for the relevance of the referring pages themselves. For example, if a highly relevant page, say $x$, refers to another page $y$, we can reasonably infer that $y$ is also relevant, even if it's referred to by just a few pages. To calculate the score of a page $i$, denoted as $r_i$, we can consider a scoring method like this:

$$ r_i = \sum_{j \rightarrow i} \frac{r_j}{d_j}$$

This implies that the relevance score of page $i$, denoted as $r_i$, is calculated as a weighted sum of the relevance scores of all pages referring to $i$. Each referring page $j$ is weighted according to its out-degree $d_j$, representing the number of pages referred from $j$.. However, without knowing the relevance of all pages referring to $i$, we cannot accurately determine the relevance of $i$. Let's attempt to formalize what we know for the previous graph:

$$
\begin{alignat*}{4}
    r_0 =& \frac{r_4}{3}\\
    r_1 =& \frac{r_2}{2} + \frac{r_4}{3} + r_3\\
    r_2 =& \frac{r_0}{3} + \frac{r_4}{3}\\
    r_3 =& \frac{r_2}{2} + \frac{r_0}{3}\\
    r_4 =& \frac{r_0}{3} + r_1
\end{alignat*}
$$

In matrix form:

$$
P =\begin{bmatrix}
0 & 0 & 0 & 0 & \frac{1}{3}\\
0 & 0 & \frac{1}{2} & 1 & \frac{1}{3}\\
\frac{1}{3} & 0 & 0 & 0 & \frac{1}{3}\\
\frac{1}{3} & 0 & \frac{1}{2} & 0 & 0\\
\frac{1}{3} & 1 & 0 & 0 & 0 \\
\end{bmatrix}
$$

We have good news: this matrix is sparse, meaning it contains many zeros. This sparsity allows us to store only the non-zero elements, saving storage space. Additionally, the matrix's transpose, denoted by $P^T$, is row-stochastic. This means that each row in $P^T$ sums to 1 because each column in the original matrix, $P$, represents a probability distribution.

This matrix, represents the transition probabilities between web pages in a simplified web graph. Each element, $P_{ij}$, signifies the probability of a random walker moving from web page $i$ to web page $j$ in a single step.

Consider the first column, specifically the elements $P_{0,j}$ (where j represents any web page):

* $P_{0,2}$: Probability of the random walker transitioning from page 0 to page 2 in one step.
* $P_{0,3}$: Probability of the random walker transitioning from page 0 to page 3 in one step.
* $P_{0,4}$: Probability of the random walker transitioning from page 0 to page 4 in one step.

In the provided example, these probabilities are $\frac{1}{3}$, $\frac{1}{3}$, and $\frac{1}{3}$, respectively. This indicates that if the random walker starts at page 0, it has an equal chance (one-third) of transitioning to any of the pages 2, 3, and 4 in the next step. It's important to remember that random walks follow the defined connections in the graph, and the walker doesn't "jump" to any arbitrary page but moves based on the transition probabilities between connected pages.


Let's suppose our random walker can start from any node, so its initial probability distribution is: 

$$
\pi = \begin{bmatrix}
\frac{1}{5} & \frac{1}{5} & \frac{1}{5} & \frac{1}{5} & \frac{1}{5}\\
\end{bmatrix}
$$

Let's ask ourselves, what is the probability of a random walker with initial distribution $\pi$ of ending in any other page after one step? Actually, let's begin with a simple question what is the probability of ending the state $5$. 

- We have a probability of $\frac{1}{5}$ of starting on $0$ and probability $\frac{1}{3}$ of moving to $4$ ($\frac{1}{5}\frac{1}{3}=\frac{1}{15}$).
- We have a probability of $\frac{1}{5}$ of starting on $1$ and probability $1$ of moving to $4$ ($\frac{1}{5}1=\frac{1}{5}$).
- We have a probability of $\frac{1}{5}$ of starting on $2$ and probability $0$ of moving to $4$ ($\frac{1}{5}0=0$).
- We have a probability of $\frac{1}{5}$ of starting on $3$ and probability $0$ of moving to $4$ ($\frac{1}{5}0=0$).
- We have a probability of $\frac{1}{5}$ of starting on $4$ and probability $0$ of moving to $4$ ($\frac{1}{5}0=0$).

So, we have a probability of ending in $4$ of $\frac{1}{15} + \frac{1}{5} + 0 + 0 + 0 = \frac{4}{15}$. This amount to multiply the last row of $P$ with $\pi$:

$$
P_4\pi = \begin{bmatrix}
\frac{1}{3} & 1 & 0 & 0 & 0\\
\end{bmatrix}
\begin{bmatrix}
\frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5}\\
\end{bmatrix} = \frac{4}{15}
$$

In practice, we can obtain the probability of ending in any state after one step, $\pi^{(1)}$, by multiplying $P$ and $\pi$.

$$
P\pi = \pi^{(1)} =\begin{bmatrix}
0 & 0 & 0 & 0 & \frac{1}{3}\\
0 & 0 & \frac{1}{2} & 1 & \frac{1}{3}\\
\frac{1}{3} & 0 & 0 & 0 & \frac{1}{3}\\
\frac{1}{3} & 0 & \frac{1}{2} & 0 & 0\\
\frac{1}{3} & 1 & 0 & 0 & 0 \\
\end{bmatrix}
\begin{bmatrix}
\frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5} \\ \frac{1}{5}\\
\end{bmatrix}= 
\begin{bmatrix}
\frac{1}{15} & \frac{11}{30} & \frac{2}{15} & \frac{1}{6} & \frac{4}{15}\\
\end{bmatrix}
$$

We can obtain the probability of a random walker finding himself on any webpage after two steps, $\pi^{(2)}$, by multiplying $P$ and $\pi^{(1)}$. Through an iterative process, we can obtain the probability of ending on any page after any finite number of steps. 

______________________
## PageRank on spark

In [ ]:
# load spark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName("Python Spark SQL basic example") \
                    .getOrCreate()
spark

In [ ]:
# First let's try an example 
dataset = spark.sparkContext.parallelize([(0,3), (0,2), (0,4), 
                                          (1,4),
                                          (2,1), (2,3),
                                          (3,1),
                                          (4,0), (4,1), (4,2)])

In [ ]:
# let's peek the first entries
dataset.collect()

In [ ]:
total_pages = dataset.keys().distinct().count()
total_pages

### Exercises
* (⭐) process `dataset` to return an rdd with nodes associated to the directly reachable nodes (e.g. `(0,[2,3,4])`)
* (⭐) process `dataset` to return an rdd with nodes associated to the number of directly reachable nodes (e.g. `(0,3)`)

In [ ]:
# compute the out-degree for each node
id2degree = dataset.countByKey()
id2degree[0],id2degree[1],id2degree[2]

In [ ]:
# Now we append edges the transition probabilities
P = dataset.map(lambda x:(x[0],x[1],1/id2degree[x[0]]))
P.take(20)

In [ ]:
# let's set the intial probability distribution
# Here, each node has equal probability
import numpy as np
p = np.array([1/total_pages for i in range(total_pages)])
p

Now we need to implement the distributed version of the matrix multiplication. This part will be a little tricky. We will assume that the vector $p$ can fit in memory. 
- The matrix $P$ is represented as $(i,j,m_{ij})$.
- The vector $\pi$ is represented as $(j, v_j)$

The algorithm proceed as follows:
- Firstly, we map each $(i,j,m_{ij}) \rightarrow (i, m_{ij}v_j)$
- Next, we reduce by key $(i, [m_{ij}v_j, \dots, m_{it}v_t]) \rightarrow (i, m_{ij}v_j + \dots + m_{it}v_t)$

$$P\pi = y$$
$$y_i = \sum_k m_{ik}\pi_k$$


In [ ]:
# columns do sum to 1.
PT = P.map(lambda x: (x[1],x[0],x[2]))
PT  .map(lambda x: (x[1],x[2]))\
    .reduceByKey(lambda x,y: x+y)\
    .take(10)
    


In [ ]:

for i in range(10):
    # P x p
    p_rdd = PT.map(lambda x:(x[0],(x[2]*p[x[1]])))\
           .reduceByKey(lambda x,y: x+y)\
           .collect()
    # update p
    for idx,prb in p_rdd:
        p[idx] = prb
     
    print(f"iteration {i}, current p: {p}")


In [ ]:
list(zip(p.argsort()[::-1], p[p.argsort()[::-1]]))

### On real data

Let's delve into a real dataset sourced from https://www.cs.cornell.edu/courses/cs685/2002fa/. You can access the raw file directly from https://www.cs.cornell.edu/courses/cs685/2002fa/data/gr0.California. These web subgraphs were generated by expanding a 200-page response set to a search engine query, much like in the hub/authority algorithm. It's worth noting that this data was collected some time ago, so it's expected that some of the links may be broken.

Before proceeding, let's attempt to implement the PageRank algorithm ourselves.


In [ ]:
## let's download the dataset
import wget, os
if not os.path.isfile("dataset.txt"):
    wget.download(url = "https://www.cs.cornell.edu/courses/cs685/2002fa/data/gr0.California", out = "dataset.txt")

In [ ]:
# load the dataset
from pprint import pprint
dataset = spark.sparkContext.textFile(name = "dataset.txt")

# let's have a peek a our dataset
print("dataset --->")
pprint(dataset.take(10))

In [ ]:
# get nodes
id2ref = dataset.filter(lambda x:x.startswith("n"))\
                .map(lambda x:tuple(x.split(" ")))\
                .map(lambda x:(int(x[1]),x[2]))
print("firsts 10 nodes entries: ", id2ref.take(10),end="\n\n")

# get edges
id2id = dataset.filter(lambda x:x.startswith("e"))\
               .map(lambda x:tuple(x.split(" ")))\
               .map(lambda x:(int(x[1]),int(x[2])))
print("firsts 10 edges entries: ", id2id.take(10),end="\n\n")


Next, we need to address a significant issue. 

Some nodes do not appear among the edges in our dataset. Consequently, the stochastic matrix would contain some rows entirely filled with zeros. This poses a problem because when multiplying the transition matrix $P$ by the probability vector $\pi$, we cannot expect a valid probability distribution.

There are several methods to tackle this issue, the simplest solution is simply to remove nodes that do not have edges.

In [ ]:
hidden = max(id2id.keys().max(), id2id.values().max()) + 1
id2hidden = spark.sparkContext.parallelize([(i,hidden) for i in range(hidden)])
hidden2id = spark.sparkContext.parallelize([(hidden,i) for i in range(hidden)])
id2id_filled  = id2id.union(id2hidden).union(hidden2id)
id2ref_filled = id2ref.union(spark.sparkContext.parallelize([(hidden, "None")]))

In [ ]:
# compute the out-degree for each node
id2degree = id2id_filled.countByKey()
print(f"degree of node 0:{id2degree[0]}, degree of node 1:{id2degree[1]}, degree of node 2:{id2degree[2]}.\n")

In [ ]:
# compute sparse transition matrix
P = id2id_filled.map(lambda x:(x[0],x[1],1/id2degree[x[0]]))
PT = P.map(lambda x: (x[1],x[0],x[2]))
print("firsts 10 matrix entries:", P.take(10), end="\n\n")
    
# compute total number of nodes
connected_nodes = id2id.keys().union(id2id.values()).distinct().count()
total_nodes = id2ref_filled.map(lambda x:x[0]).count()
print(f"total nodes: {total_nodes}, connected_nodes: {connected_nodes}\n")

# compute probabilities vector
import numpy as np
p = np.full((total_nodes,), 1/(total_nodes))

# P*p for some iteration
for i in range(10):
    new_p = PT.map(lambda x:(x[0],(x[2]*p[x[1]])))\
              .reduceByKey(lambda x,y: x+y)\
              .collect()
    for idx,prb in new_p:
        p[idx] = prb

print(p.sum())

# print top pages
for page in list(zip(p.argsort()[::-1], p[p.argsort()[::-1]]))[:10]:
    print(f"prb:{page[1]}, page:{id2ref_filled.lookup(page[0])}")



In [ ]:
p.sum()

### Exercises
* (⭐⭐⭐) modify this algorithm: remove the unreferenced nodes and recompute the random walk probabilities